In [7]:

# things we need for NLP
 
import nltk

from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()



# things we need for Tensorflow

import numpy as np

import tflearn

import tensorflow as tf

import random

In [8]:

# import our chat-bot intents file

import json

with open('C://Users//joshg//OneDrive//Desktop//chatbot.json') as json_data:

    intents = json.load(json_data)

In [37]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'bicycles',
   'patterns': ['Which bicycles do you have?',
    'What kinds of bicycles are there?',
    'What do you rent?'],
   'responses': ['We rent Hercules, BSA and Firefox bicycle models',
    'We have Hercules, BSA and Firefox brands']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
   

In [38]:
words = []

classes = []

documents = []

ignore_words = ['?']

# loop through each sentence in our intents patterns

for intent in intents['intents']:

    for pattern in intent['patterns']:

        # tokenize each word in the sentence

        w = nltk.word_tokenize(pattern)

        # add to our words list

        words.extend(w)

        # add to documents in our corpus

        documents.append((w, intent['tag']))

        # add to our classes list

        if intent['tag'] not in classes:

            classes.append(intent['tag'])


# stem and lower each word and remove duplicates

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]

words = sorted(list(set(words)))


# remove duplicates

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique stemmed words", words)

23 documents
7 classes ['bicycles', 'goodbye', 'greeting', 'hours', 'opentoday', 'payments', 'thanks']
38 unique stemmed words ["'s", 'acceiv', 'anyon', 'ar', 'bicyc', 'bye', 'card', 'cash', 'credit', 'day', 'do', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'is', 'kind', 'lat', 'mastercard', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'today', 'what', 'when', 'which', 'yo', 'you']


In [28]:

# create our training data

training = []

output = []

# create an empty array for our output

output_empty = [0] * len(classes)


# training set, bag of words for each sentence

for doc in documents:

    # initialize our bag of words

    bag = []

    # list of tokenized words for the pattern

    pattern_words = doc[0]

    # stem each word

    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    # create our bag of words array

    for w in words:

        bag.append(1) if w in pattern_words else bag.append(0)


    # output is a '0' for each tag and '1' for current tag

    output_row = list(output_empty)

    output_row[classes.index(doc[1])] = 1



    training.append([bag, output_row])


# shuffle our features and turn into np.array

random.shuffle(training)

training = np.array(training)


# create train and test lists

train_x = list(training[:,0])

train_y = list(training[:,1])

# 

In [39]:
# reset underlying graph data

tf.reset_default_graph()

# Build neural network

net = tflearn.input_data(shape=[None, len(train_x[0])])

net = tflearn.fully_connected(net, 8)

net = tflearn.fully_connected(net, 8)

net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')

net = tflearn.regression(net)



# Define model and setup tensorboard

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training (apply gradient descent algorithm)

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)

model.save('model.tflearn')

Training Step: 2999  | total loss: 0.02702 | time: 0.005s
| Adam | epoch: 1000 | loss: 0.02702 - acc: 0.9998 -- iter: 16/23
Training Step: 3000  | total loss: 0.02593 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.02593 - acc: 0.9998 -- iter: 23/23
--
INFO:tensorflow:C:\Users\joshg\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [29]:
import pickle

pickle.dump( {'words':words, 'classes':classes, 'train_x'
              :train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [15]:
# restore all of our data structures

import pickle

data = pickle.load( open( "training_data", "rb" ) )

words = data['words']

classes = data['classes']

train_x = data['train_x']

train_y = data['train_y']

In [17]:

def clean_up_sentence(sentence):

    # tokenize the pattern

    sentence_words = nltk.word_tokenize(sentence)

    # stem each word

    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words



# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=False):

    # tokenize the pattern

    sentence_words = clean_up_sentence(sentence)

    # bag of words

    bag = [0]*len(words)  

    for s in sentence_words:

        for i,w in enumerate(words):

            if w == s: 

                bag[i] = 1

                if show_details:

                    print ("found in bag: %s" % w)



    return(np.array(bag))

In [32]:


ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [40]:
classify('is your shop open today?')

[('opentoday', 0.6736991), ('thanks', 0.3262534)]

In [41]:
response('is your shop open today?')

We're open every day from 9am-9pm


In [42]:
response('do you take cash?')

We accept most major credit cards


In [36]:
response("Goodbye, see you later")

Bye! Come back again soon.


In [21]:
response("Is anyone there?")

Hi there, how can I help?
